# Play with the data

    
1. 1st step: Crawl the data with Python Beautiful Soup (optional, you will get ~10% of bonus mark for the proposal if you do it)
2. 2nd step: Crawl the data with SPACEX API (optional, you will get ~15% of bonus mark for the proposal if you do it. Bonus marks are accumulable but the maximum mark you get is 100%.)
3. 3rd step: Download an example crawled data for past SPACEX launches using SPACEX API.
4. 4th step: Data wrangling for the example crawled data – filter data for Falcon 9, deal with missing values (for payload mass).
5. Next steps: further understanding the data, exploration data analysis and interactive visualization of the data. They will be discussed in next Friday lecture.
6. Future steps: preparing data for machine learning models, predictive analysis. 



This notebook collated my work done so far with note and explanation.Others are the initial notebooks with many attempts and it could be difficlut to read and run.

In [1]:
import requests
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

### 1.  Dataset2

use ***requests.get*** for Rest API

In [2]:
url = 'https://api.spacexdata.com/v4/launchpads'
response = requests.get(url)

In [3]:
print(response.json())

[{'images': {'large': ['https://i.imgur.com/7uXe1Kv.png']}, 'name': 'VAFB SLC 3W', 'full_name': 'Vandenberg Space Force Base Space Launch Complex 3W', 'locality': 'Vandenberg Space Force Base', 'region': 'California', 'latitude': 34.6440904, 'longitude': -120.5931438, 'launch_attempts': 0, 'launch_successes': 0, 'rockets': ['5e9d0d95eda69955f709d1eb'], 'timezone': 'America/Los_Angeles', 'launches': [], 'status': 'retired', 'details': "SpaceX's original west coast launch pad for Falcon 1. It was used in a static fire test but was never employed for a launch, and was abandoned due to range scheduling conflicts arising from overflying other active pads.", 'id': '5e9e4501f5090910d4566f83'}, {'images': {'large': ['https://i.imgur.com/9oEMXwa.png']}, 'name': 'CCSFS SLC 40', 'full_name': 'Cape Canaveral Space Force Station Space Launch Complex 40', 'locality': 'Cape Canaveral', 'region': 'Florida', 'latitude': 28.5618571, 'longitude': -80.577366, 'launch_attempts': 95, 'launch_successes': 93,

use pandaas to get the data

In [4]:
df_v4 = pd.read_json('https://api.spacexdata.com/v4/launches/past')
df_v5 = pd.read_json('https://api.spacexdata.com/v5/launches')
df_launchpads = pd.read_json('https://api.spacexdata.com/v4/launchpads')

In [7]:
df_v4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fairings               145 non-null    object 
 1   links                  179 non-null    object 
 2   static_fire_date_utc   121 non-null    object 
 3   static_fire_date_unix  121 non-null    float64
 4   net                    179 non-null    bool   
 5   window                 117 non-null    float64
 6   rocket                 179 non-null    object 
 7   success                178 non-null    float64
 8   failures               179 non-null    object 
 9   details                134 non-null    object 
 10  crew                   179 non-null    object 
 11  ships                  179 non-null    object 
 12  capsules               179 non-null    object 
 13  payloads               179 non-null    object 
 14  launchpad              179 non-null    object 
 15  flight

In [8]:
df_v5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fairings               154 non-null    object 
 1   links                  189 non-null    object 
 2   static_fire_date_utc   121 non-null    object 
 3   static_fire_date_unix  121 non-null    float64
 4   net                    189 non-null    bool   
 5   window                 117 non-null    float64
 6   rocket                 189 non-null    object 
 7   success                178 non-null    float64
 8   failures               189 non-null    object 
 9   details                134 non-null    object 
 10  crew                   189 non-null    object 
 11  ships                  189 non-null    object 
 12  capsules               189 non-null    object 
 13  payloads               189 non-null    object 
 14  launchpad              189 non-null    object 
 15  flight

##### Changes from v4 -> v5
Crew is now an array of objects, to allow for more data on an individual launch for a crew member

In [9]:
df_launchpads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   images            6 non-null      object 
 1   name              6 non-null      object 
 2   full_name         6 non-null      object 
 3   locality          6 non-null      object 
 4   region            6 non-null      object 
 5   latitude          6 non-null      float64
 6   longitude         6 non-null      float64
 7   launch_attempts   6 non-null      int64  
 8   launch_successes  6 non-null      int64  
 9   rockets           6 non-null      object 
 10  timezone          6 non-null      object 
 11  launches          6 non-null      object 
 12  status            6 non-null      object 
 13  details           6 non-null      object 
 14  id                6 non-null      object 
dtypes: float64(2), int64(2), object(11)
memory usage: 848.0+ bytes


In [10]:
df_v5.head()

,fairings,links,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,...,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id
0,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,...,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a
1,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,...,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",True,False,None,5eb87cdaffd86e000604b32b
2,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 140, 'altitude': 35, 'reason': 'resi...",Residual stage 1 thrust led to collision betwe...,...,2008-08-03T03:34:00.000Z,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight'...",True,False,None,5eb87cdbffd86e000604b32c
3,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,1.0,[],Ratsat was carried to orbit on the first succe...,...,2008-09-28T23:15:00.000Z,1222643700,2008-09-28T11:15:00+12:00,hour,False,"[{'core': '5e9e289ef3591855dc3b2626', 'flight'...",True,False,None,5eb87cdbffd86e000604b32d
4,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,1.0,[],None,...,2009-07-13T03:35:00.000Z,1247456100,2009-07-13T15:35:00+12:00,hour,False,"[{'core': '5e9e289ef359184f103b2627', 'flight'...",True,False,None,5eb87cdcffd86e000604b32e


##### We could match the launchpads and launches by launchpad id, others have the similar map from launch data to core/payload/rocket data

In [17]:
example = pd.read_csv('dataset_space_API_past_launches_raw.csv')
example.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,Falcon 1,20.0,LEO,Kwajalein Atoll,None None,1,False,False,False,NaN,NaN,0,Merlin1A,167.743129,9.047721
1,2,2007-03-21,Falcon 1,NaN,LEO,Kwajalein Atoll,None None,1,False,False,False,NaN,NaN,0,Merlin2A,167.743129,9.047721
2,4,2008-09-28,Falcon 1,165.0,LEO,Kwajalein Atoll,None None,1,False,False,False,NaN,NaN,0,Merlin2C,167.743129,9.047721
3,5,2009-07-13,Falcon 1,200.0,LEO,Kwajalein Atoll,None None,1,False,False,False,NaN,NaN,0,Merlin3C,167.743129,9.047721
4,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857


In [19]:
example.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   FlightNumber    94 non-null     int64  
 1   Date            94 non-null     object 
 2   BoosterVersion  94 non-null     object 
 3   PayloadMass     88 non-null     float64
 4   Orbit           94 non-null     object 
 5   LaunchSite      94 non-null     object 
 6   Outcome         94 non-null     object 
 7   Flights         94 non-null     int64  
 8   GridFins        94 non-null     bool   
 9   Reused          94 non-null     bool   
 10  Legs            94 non-null     bool   
 11  LandingPad      64 non-null     object 
 12  Block           90 non-null     float64
 13  ReusedCount     94 non-null     int64  
 14  Serial          94 non-null     object 
 15  Longitude       94 non-null     float64
 16  Latitude        94 non-null     float64
dtypes: bool(3), float64(4), int64(3), obj

schema shows on github https://github.com/r-spacex/SpaceX-API/blob/master/docs/launches/v5/schema.md

In [24]:
df_rockets=pd.read_json('https://api.spacexdata.com/v4/rockets')
df_payloads = pd.read_json("https://api.spacexdata.com/v4/payloads")
df_cores = pd.read_json('https://api.spacexdata.com/v4/cores')

### 1. get luanch info
#### 1.1 split core from launch data

In [25]:
df_coreinfo = pd.DataFrame(columns=['core','flight','gridfins','reused','legs','landing_type','landing_success','landpad'
    ])
df_coreinfo

,core,flight,gridfins,reused,legs,landing_type,landing_success,landpad


In [26]:
for coreinfo in df_v5['cores']:
    core = coreinfo[0]['core']
    flight = coreinfo[0]['flight']
    gridfins = coreinfo[0]['gridfins']
    reused = coreinfo[0]['reused']
    legs = coreinfo[0]['legs']
    landing_type = coreinfo[0]['landing_type']
    landing_success = coreinfo[0]['landing_success']
    landpad = coreinfo[0]['landpad']
    df_coreinfo = df_coreinfo.append({'core':core,'flight':flight,
                                      'gridfins':gridfins,'reused':reused,
                                      'legs':legs,'landing_type':landing_type,
                                      'landing_success':landing_success,
                                      'landpad':landpad
    },ignore_index=True)
    

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_coreinfo = df_coreinfo.append({'core':core,'flight':flight,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_coreinfo = df_coreinfo.append({'core':core,'flight':flight,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_coreinfo = df_coreinfo.append({'core':core,'flight':flight,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_coreinfo = df_coreinfo.append({'core':core,'flight':flight,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_coreinfo = df_coreinfo.append({'core':core,'flight':flight,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_coreinfo = df_coreinfo.append({'core':core,'flight':flight,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_coreinfo = df_coreinfo.append({'core':core,'flight':flight,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_coreinfo = df_coreinfo.append({'core':core,'flight':flight,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_coreinfo = df_coreinfo.append({'core':core,'flight':flight,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_coreinfo = df_coreinfo.append({'core':core,'flight':flight,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_coreinfo = df_coreinfo.append({'core':core,'flight':flight,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_coreinfo = df_coreinfo.append({'core':core,'flight':flight,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_coreinfo = df_coreinfo.append({'core':core,'flight':flight,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_coreinfo = df_coreinfo.append({'core':core,'flight':flight,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_coreinfo = df_coreinfo.append({'core':core,'flight':flight,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\618275585.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

In [27]:
df_coreinfo.head()

,core,flight,gridfins,reused,legs,landing_type,landing_success,landpad
0,5e9e289df35918033d3b2623,1,False,False,False,None,None,None
1,5e9e289ef35918416a3b2624,1,False,False,False,None,None,None
2,5e9e289ef3591814873b2625,1,False,False,False,None,None,None
3,5e9e289ef3591855dc3b2626,1,False,False,False,None,None,None
4,5e9e289ef359184f103b2627,1,False,False,False,None,None,None


#### 1.2. split multi payload

In [30]:
df_payloadid = pd.DataFrame(columns=['payload'])
df_payloadid

,payload


In [31]:

for payload in df_v5_clean1['payloads']:
    if payload == [] :
        payload_id = None
    elif len(payload) == 1:
        payload_id = payload[0]
    else:
        payload_id = 'Multi'
#     print(payload_id)
    df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_payloadid = df_payloadid.append({'payload':payload_id},ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1482379514.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

In [34]:
df_payloadid

,payload
0,5eb0e4b5b6c3bb0006eeb1e1
1,5eb0e4b6b6c3bb0006eeb1e2
2,Multi
3,5eb0e4b7b6c3bb0006eeb1e5
4,5eb0e4b7b6c3bb0006eeb1e6
...,...
184,None
185,6243b896af52800c6e91926e
186,5fe3b86eb3467846b324217c
187,6243b788af52800c6e91926b


#### 1.3. other info of launch

In [28]:
df_v5_clean1 = df_v5[['flight_number','payloads','launchpad','rocket','date_local']]
df_v5_clean1

,flight_number,payloads,launchpad,rocket,date_local
0,1,[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,5e9d0d95eda69955f709d1eb,2006-03-25T10:30:00+12:00
1,2,[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,5e9d0d95eda69955f709d1eb,2007-03-21T13:10:00+12:00
2,3,"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006e...",5e9e4502f5090995de566f86,5e9d0d95eda69955f709d1eb,2008-08-03T15:34:00+12:00
3,4,[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,5e9d0d95eda69955f709d1eb,2008-09-28T11:15:00+12:00
4,5,[5eb0e4b7b6c3bb0006eeb1e6],5e9e4502f5090995de566f86,5e9d0d95eda69955f709d1eb,2009-07-13T15:35:00+12:00
...,...,...,...,...,...
184,186,[],5e9e4501f509094ba4566f84,5e9d0d95eda69973a809d1ec,2022-08-31T20:00:00-04:00
185,198,[6243b896af52800c6e91926e],5e9e4501f509094ba4566f84,5e9d0d95eda69973a809d1ec,2022-11-30T19:00:00-05:00
186,200,[5fe3b86eb3467846b324217c],5e9e4502f509094188566f88,5e9d0d95eda69974db09d1ed,2022-11-30T19:00:00-05:00
187,202,[6243b788af52800c6e91926b],5e9e4501f509094ba4566f84,5e9d0d95eda69973a809d1ec,2022-11-30T19:00:00-05:00


#### 1.4 get the clean launch

In [33]:
df_clean_launch = pd.concat([df_v5_clean1,df_coreinfo,df_payloadid],axis=1)
df_clean_launch

,flight_number,payloads,launchpad,rocket,date_local,core,flight,gridfins,reused,legs,landing_type,landing_success,landpad,payload
0,1,[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,5e9d0d95eda69955f709d1eb,2006-03-25T10:30:00+12:00,5e9e289df35918033d3b2623,1,False,False,False,None,None,None,5eb0e4b5b6c3bb0006eeb1e1
1,2,[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,5e9d0d95eda69955f709d1eb,2007-03-21T13:10:00+12:00,5e9e289ef35918416a3b2624,1,False,False,False,None,None,None,5eb0e4b6b6c3bb0006eeb1e2
2,3,"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006e...",5e9e4502f5090995de566f86,5e9d0d95eda69955f709d1eb,2008-08-03T15:34:00+12:00,5e9e289ef3591814873b2625,1,False,False,False,None,None,None,Multi
3,4,[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,5e9d0d95eda69955f709d1eb,2008-09-28T11:15:00+12:00,5e9e289ef3591855dc3b2626,1,False,False,False,None,None,None,5eb0e4b7b6c3bb0006eeb1e5
4,5,[5eb0e4b7b6c3bb0006eeb1e6],5e9e4502f5090995de566f86,5e9d0d95eda69955f709d1eb,2009-07-13T15:35:00+12:00,5e9e289ef359184f103b2627,1,False,False,False,None,None,None,5eb0e4b7b6c3bb0006eeb1e6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,186,[],5e9e4501f509094ba4566f84,5e9d0d95eda69973a809d1ec,2022-08-31T20:00:00-04:00,None,None,None,None,None,None,None,None,None
185,198,[6243b896af52800c6e91926e],5e9e4501f509094ba4566f84,5e9d0d95eda69973a809d1ec,2022-11-30T19:00:00-05:00,None,None,None,None,None,None,None,None,6243b896af52800c6e91926e
186,200,[5fe3b86eb3467846b324217c],5e9e4502f509094188566f88,5e9d0d95eda69974db09d1ed,2022-11-30T19:00:00-05:00,5fe3b8f2b3467846b3242181,1,True,False,True,None,None,None,5fe3b86eb3467846b324217c
187,202,[6243b788af52800c6e91926b],5e9e4501f509094ba4566f84,5e9d0d95eda69973a809d1ec,2022-11-30T19:00:00-05:00,None,None,None,None,None,None,None,None,6243b788af52800c6e91926b


### 2. get the clean rocket info

In [36]:
df_clean_rockets = df_rockets[['id','name']]
df_clean_rockets.rename(columns={'id':'rocket','name':'rocketName'},inplace=True)
df_clean_rockets

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\193276671.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean_rockets.rename(columns={'id':'rocket','name':'rocketName'},inplace=True)


,rocket,rocketName
0,5e9d0d95eda69955f709d1eb,Falcon 1
1,5e9d0d95eda69973a809d1ec,Falcon 9
2,5e9d0d95eda69974db09d1ed,Falcon Heavy
3,5e9d0d96eda699382d09d1ee,Starship


### 3. get the clean payload info

In [35]:
df_clean_payloads = df_payloads[['id','mass_kg','orbit','name']]
df_clean_payloads.rename(columns={'id':'payload','name':'payloadName'},inplace=True)
df_clean_payloads

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\3697139516.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean_payloads.rename(columns={'id':'payload','name':'payloadName'},inplace=True)


,payload,mass_kg,orbit,payloadName
0,5eb0e4b5b6c3bb0006eeb1e1,20.0,LEO,FalconSAT-2
1,5eb0e4b6b6c3bb0006eeb1e2,NaN,LEO,DemoSAT
2,5eb0e4b6b6c3bb0006eeb1e3,NaN,LEO,Trailblazer
3,5eb0e4b6b6c3bb0006eeb1e4,NaN,LEO,PRESat
4,5eb0e4b7b6c3bb0006eeb1e5,165.0,LEO,RatSat
...,...,...,...,...
201,6278484e57b51b752c5c5a63,13260.0,VLEO,Starlink 4-19 (v1.5)
202,628026a42c0aa958437dcf7c,2608.0,BEO,Psyche
203,628027062c0aa958437dcf7d,72.0,BEO,Janus
204,62adecbcd26f4f711fa53848,700.0,LEO,Globalstar FM15


### 4. get the clean launchpad info

In [37]:
df_clean_launchpads = df_launchpads[['id','name','latitude','longitude']]
df_clean_launchpads.rename(columns={'id':'launchpad','name':'launchpadName'},inplace=True)
df_clean_launchpads

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\4152662409.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean_launchpads.rename(columns={'id':'launchpad','name':'launchpadName'},inplace=True)


,launchpad,launchpadName,latitude,longitude
0,5e9e4501f5090910d4566f83,VAFB SLC 3W,34.644090,-120.593144
1,5e9e4501f509094ba4566f84,CCSFS SLC 40,28.561857,-80.577366
2,5e9e4502f5090927f8566f85,STLS,25.997264,-97.156085
3,5e9e4502f5090995de566f86,Kwajalein Atoll,9.047721,167.743129
4,5e9e4502f509092b78566f87,VAFB SLC 4E,34.632093,-120.610829
5,5e9e4502f509094188566f88,KSC LC 39A,28.608058,-80.603956


### 5. get the clean core info

In [38]:
df_clean_cores = df_cores[['id','block','serial','reuse_count']]
df_clean_cores.rename(columns={'id':'core','name':'coreName'},inplace=True)
df_clean_cores

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\557569245.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean_cores.rename(columns={'id':'core','name':'coreName'},inplace=True)


,core,block,serial,reuse_count
0,5e9e289df35918033d3b2623,NaN,Merlin1A,0
1,5e9e289ef35918416a3b2624,NaN,Merlin2A,0
2,5e9e289ef3591814873b2625,NaN,Merlin1C,0
3,5e9e289ef3591855dc3b2626,NaN,Merlin2C,0
4,5e9e289ef359184f103b2627,NaN,Merlin3C,0
...,...,...,...,...
76,61fae5947aa67176fe3e0e1e,5.0,B1071,3
77,627843d657b51b752c5c5a53,5.0,B1072,0
78,627843db57b51b752c5c5a54,5.0,B1073,2
79,627843de57b51b752c5c5a55,5.0,B1074,0


###  6. join all 5 parts

In [40]:
df_join_1 = pd.merge(df_clean_launch,df_clean_launchpads,on='launchpad',how = 'left')
df_join_2 = pd.merge(df_join_1,df_clean_cores,on='core',how = 'left')
df_join_3 = pd.merge(df_join_2,df_clean_rockets,on='rocket',how = 'left')
df_join_4 = pd.merge(df_join_3,df_clean_payloads,on='payload',how = 'left')

In [66]:
df_join_4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189 entries, 0 to 188
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   flight_number    189 non-null    int64  
 1   payloads         189 non-null    object 
 2   launchpad        189 non-null    object 
 3   rocket           189 non-null    object 
 4   date_local       189 non-null    object 
 5   core             180 non-null    object 
 6   flight           180 non-null    object 
 7   gridfins         180 non-null    object 
 8   reused           180 non-null    object 
 9   legs             180 non-null    object 
 10  landing_type     150 non-null    object 
 11  landing_success  148 non-null    object 
 12  landpad          143 non-null    object 
 13  payload          176 non-null    object 
 14  launchpadName    189 non-null    object 
 15  latitude         189 non-null    float64
 16  longitude        189 non-null    float64
 17  block           

In [67]:
df_clean_dataset2 = df_join_4.drop(['payloads','launchpad','rocket','core'],axis=1,inplace=False)

In [69]:
dataset2_ini = df_clean_dataset2
dataset2_ini.loc[dataset2_ini['payload']=='Multi','payloadName']='Multi-payloads'
dataset2_ini.drop(['payload'],axis=1,inplace=True)
dataset2_ini.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189 entries, 0 to 188
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   flight_number    189 non-null    int64  
 1   date_local       189 non-null    object 
 2   flight           180 non-null    object 
 3   gridfins         180 non-null    object 
 4   reused           180 non-null    object 
 5   legs             180 non-null    object 
 6   landing_type     150 non-null    object 
 7   landing_success  148 non-null    object 
 8   landpad          143 non-null    object 
 9   launchpadName    189 non-null    object 
 10  latitude         189 non-null    float64
 11  longitude        189 non-null    float64
 12  block            175 non-null    float64
 13  serial           180 non-null    object 
 14  reuse_count      180 non-null    float64
 15  rocketName       189 non-null    object 
 16  mass_kg          141 non-null    float64
 17  orbit           

Outcome is a combine of landing type and landing success, thus I choose to keep it as origin.
PayloadName has info about if its multi payloads(so the payload detail is null), and I choose to keep it too.

In [70]:
dataset2 = dataset2_ini.rename(columns={'flight_number':'FlightNumber','date_local':'Date',
                                        'rocketName':'BoosterVersion','mass_kg':'PayloadMass',
                                       'orbit':'Orbit','launchpadName':'LaunchSite',
                                        'landing_type':'LandType',
                                       'landing_success':'Outcome','flight':'Flights',
                                        'gridfins':'Gridfins','reused':'Reused',
                                        'legs':'Legs','landpad':'LandingPad',
                                        'block':'Block','reuse_count':'ReusedCount',
                                        'serial':'Serial',
                                        'longitude':'Longitude','latitude':'Latitude'
                                       },inplace=False)

In [71]:
dataset2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189 entries, 0 to 188
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   FlightNumber    189 non-null    int64  
 1   Date            189 non-null    object 
 2   Flights         180 non-null    object 
 3   Gridfins        180 non-null    object 
 4   Reused          180 non-null    object 
 5   Legs            180 non-null    object 
 6   LandType        150 non-null    object 
 7   Outcome         148 non-null    object 
 8   LandingPad      143 non-null    object 
 9   LaunchSite      189 non-null    object 
 10  Latitude        189 non-null    float64
 11  Longitude       189 non-null    float64
 12  Block           175 non-null    float64
 13  Serial          180 non-null    object 
 14  ReusedCount     180 non-null    float64
 15  BoosterVersion  189 non-null    object 
 16  PayloadMass     141 non-null    float64
 17  Orbit           164 non-null    obj

Except the payloadName and LandType, other schema is the same as the example

In [72]:
dataset2.to_csv('dataset2.csv',index = None)

# 2. Dataset1

In [73]:
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
data = requests.get(url).text
soup = BeautifulSoup(data, 'html.parser')

In [78]:
for table in soup.find_all('table'):
    print(table.get('class'))

['multicol']
['multicol']
['wikitable', 'plainrowheaders', 'collapsible']
['wikitable', 'plainrowheaders', 'collapsible']
['wikitable', 'plainrowheaders', 'collapsible']
['wikitable']
['wikitable']
['wikitable']
['wikitable']
['nowraplinks', 'hlist', 'mw-collapsible', 'autocollapse', 'navbox-inner']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'mw-collapsible', 'mw-collapsed', 'navbox-inner']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'hlist', 'mw-collapsible', 'autocollapse', 'navbox-inner']
['nowraplinks', 'navbox-subgroup']


In [79]:
tables2020 = soup.find_all('table')
table2020 = soup.find_all('table', class_='wikitable plainrowheaders collapsible')

In [80]:
url1 = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches_(2010-2019)"
data1 = requests.get(url1).text
soup1 = BeautifulSoup(data1, 'html.parser')
for table1 in soup1.find_all('table'):
    print(table1.get('class'))

['multicol']
['multicol']
['wikitable', 'plainrowheaders', 'collapsible']
['wikitable', 'plainrowheaders', 'collapsible']
['wikitable', 'plainrowheaders', 'collapsible']
['wikitable', 'plainrowheaders', 'collapsible']
['wikitable', 'plainrowheaders', 'collapsible']
['wikitable', 'plainrowheaders', 'collapsible']
['wikitable', 'plainrowheaders', 'collapsible']
['nowraplinks', 'hlist', 'mw-collapsible', 'autocollapse', 'navbox-inner']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'mw-collapsible', 'mw-collapsed', 'navbox-inner']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'hlist', 'mw-collapsible', 'autocollapse', 'navbox-inner']
['nowraplinks', 'navbox-subgroup']


In [82]:
tables2010 = soup1.find_all('table')
table2010 = soup1.find_all('table', class_='wikitable plainrowheaders collapsible')

In [83]:
table_2010_2013 = tables2010[2]
table_2014 = tables2010[3]
table_2015 = tables2010[4]
table_2016 = tables2010[5]
table_2017 = tables2010[6]
table_2018 = tables2010[7]
table_2019 = tables2010[8]
table_2020 = tables2020[2]
table_2021 = tables2020[3]
table_2022 = tables2020[4]

In [108]:
df_wikiinfo = pd.DataFrame(columns=['date_and_time','version_booster','launch_site',
                                'payload','payload_mass','orbit','customer',
                               'launch_outcome','booster_landing','description'])
df_wikiinfo

,date_and_time,version_booster,launch_site,payload,payload_mass,orbit,customer,launch_outcome,booster_landing,description


In [85]:
date_and_time = ''
version_booster = ''
launch_site = ''
payload = ''
payload_mass = ''
orbit = ''
customer = ''
launch_outcome = ''
booster_landing = ''
description = ''

### 1. data from 2010 - 2019

In [109]:
for i in range(2,9):
    for row in tables2010[i].tbody.find_all('tr'):
        colunms = row.find_all('td')
        if (len(colunms) == 9) :
            # the regular table
            date_and_time = colunms[0].text.strip()
            version_booster = colunms[1].text.strip()
            launch_site = colunms[2].text.strip()
            payload = colunms[3].text.strip()
            payload_mass = colunms[4].text.strip()
            orbit = colunms[5].text.strip()
            customer = colunms[6].text.strip()
            launch_outcome = colunms[7].text.strip()
            booster_landing = colunms[8].text.strip()
        if (len(colunms) == 1) :
            # the description of the fight, part of the former line
            description = colunms[0].text.strip()
            df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
                                      'version_booster':version_booster,
                                      'launch_site':launch_site,
                                      'payload':payload,
                                      'payload_mass':payload_mass,
                                      'orbit':orbit,
                                      'customer':customer,
                                      'launch_outcome':launch_outcome,
                                      'booster_landing':booster_landing,
                                      'description':description},ignore_index=True)



C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1986532609.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1986532609.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1986532609.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1986532609.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1986532609.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1986532609.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1986532609.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\1986532609.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

### 2. data from 2020 - now

In [112]:
for i in range(2,5):
    for row in tables2020[i].tbody.find_all('tr'):
        colunms = row.find_all('td')
        if (len(colunms) == 9) :
            # the regular table
            date_and_time = colunms[0].text.strip()
            version_booster = colunms[1].text.strip()
            launch_site = colunms[2].text.strip()
            payload = colunms[3].text.strip()
            payload_mass = colunms[4].text.strip()
            orbit = colunms[5].text.strip()
            customer = colunms[6].text.strip()
            launch_outcome = colunms[7].text.strip()
            booster_landing = colunms[8].text.strip()
        if (len(colunms) == 1) :
            # the description of the fight, part of the former line
            description = colunms[0].text.strip()
            df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
                                      'version_booster':version_booster,
                                      'launch_site':launch_site,
                                      'payload':payload,
                                      'payload_mass':payload_mass,
                                      'orbit':orbit,
                                      'customer':customer,
                                      'launch_outcome':launch_outcome,
                                      'booster_landing':booster_landing,
                                      'description':description},ignore_index=True)



C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\280437217.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\280437217.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\280437217.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\280437217.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\280437217.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\280437217.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\280437217.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\280437217.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\280437217.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\280437217.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\280437217.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_wikiinfo = df_wikiinfo.append({'date_and_time':date_and_time,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40116\280437217.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

In [113]:
df_wikiinfo.to_csv('df_wokiinfo.csv',index=False)

In [114]:
df_wikiinfo

,date_and_time,version_booster,launch_site,payload,payload_mass,orbit,customer,launch_outcome,booster_landing,description
0,"4 June 2010,18:45",F9 v1.0[2]B0003[3],"CCAFS,SLC-40",Dragon Spacecraft Qualification Unit,No payload (excl. Dragon Mass),LEO,SpaceX,Success,Failure[4][5](parachute),First flight of Falcon 9 v1.0.[6] Used a boile...
1,"8 December 2010,15:43[8]",F9 v1.0[2]B0004[3],"CCAFS,SLC-40",SpaceX COTS Demo Flight 1(Dragon C101),Unknown (excl. Dragon Mass),LEO (ISS),NASA (COTS)\nvarious others[9],Success[4],Failure[4][10](parachute),"Maiden flight of SpaceX's Dragon capsule, cons..."
2,"22 May 2012,07:44[13]",F9 v1.0[2]B0005[3],"CCAFS,SLC-40",SpaceX COTS Demo Flight 2[14](Dragon C102),"525 kg (1,157 lb)[15] (excl. Dragon mass)",LEO (ISS),NASA (COTS),Success[16],No attempt,The Dragon spacecraft demonstrated a series of...
3,"8 October 2012,00:35[17]",F9 v1.0[2]B0006[3],"CCAFS,SLC-40",SpaceX CRS-1[18](Dragon C103),"4,700 kg (10,400 lb) (excl. Dragon mass)",LEO (ISS),NASA (CRS),Success,No attempt,"CRS-1 was successful, but the secondary payloa..."
4,"1 March 2013,15:10",F9 v1.0[2]B0007[3],"CCAFS,SLC-40",SpaceX CRS-2[18](Dragon C104),"4,877 kg (10,752 lb) (excl. Dragon mass)",LEO (ISS),NASA (CRS),Success,No attempt,Last launch of the original Falcon 9 v1.0 laun...
...,...,...,...,...,...,...,...,...,...,...
169,22 July 202217:39[269],F9 B5♺B1071.4,"VSFB,SLC-4E",Starlink Group 3-2 (46 satellites)[357],"~14,100 kg (31,100 lb)",SSO,SpaceX,Success,Success(drone ship),A west-coast Starlink network launch for 97.6°...
170,24 July 202213:38[358],F9 B5 ♺B1062.8,"KSC,LC-39A",Starlink Group 4-25 (53 satellites),"~16,250 kg (35,830 lb)",LEO,SpaceX,Success,Success(drone ship),An east-coast Starlink network launch for 53.2...
171,4 August 202223:08[359],F9 B5 ♺B1052.6,"CCSFS,SLC-40",Danuri (Korea Pathfinder Lunar Orbiter)[360],"~679 kg (1,497 lb)[361]",Ballistic Lunar Transfer (BLT),KARI,Success,Success(drone ship),"South Korea's first lunar mission. In 2018, Te..."
172,10 August 202202:14[365],F9 B5♺B1073.3[366],"KSC,LC-39A",Starlink Group 4-26[367] (52 satellites),"~16,000 kg (35,000 lb)",LEO,SpaceX,Success,Success(drone ship),An east-coast Starlink network launch for 53.2...
